In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
from datetime import datetime

In [ ]:
#pip install datasets

In [ ]:
import numpy as np
import torch
import torch.utils.data
import scipy
import torch.nn.functional as F
#import datasets
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
#datasets.logging.set_verbosity_error()

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [ ]:
import tensorflow
tensorflow.random.set_seed(0)
np.random.seed(0)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/My Drive/thesis/dataset/final_dataset_with_all_stock.csv [Final one]')

In [ ]:
df = df.drop(df.columns[0],axis=1)

In [ ]:

#df

In [ ]:
df = df.sort_values(by=['stock_symbol', 'Date'])

In [ ]:
#df

In [ ]:
def calculate_price_change(group):
    Price_Change = group['Open'].shift(-1) - group['Open']
    group['price_change'] = Price_Change
    group['Open_t+1'] = group['Open'].shift(-1)
    return group

group_df = df.groupby('stock_symbol', group_keys=False).apply(calculate_price_change)


In [ ]:
def calculate_direction(group):
    Direction = group['price_change'].apply(lambda x: 1 if isinstance(x, (float, int)) and x > 0 else 0 if isinstance(x, (float, int)) and x <= 0 else None)
    group['direction'] = Direction
    return group

group_df = group_df.groupby('stock_symbol', group_keys=False).apply(calculate_direction)


In [ ]:
#group_df

In [ ]:
group_df.isnull().sum()

stock_symbol                    0
Date                            0
Open                            0
Close                           0
High                            0
Low                             0
Volume                          0
macd                            0
tweet_sentiment_count_0         0
tweet_sentiment_count_1         0
tweet_sentiment_count_2         0
tweet_sentiment_mean            0
stocktwit_sentiment_count_0     0
stocktwit_sentiment_count_1     0
stocktwit_sentiment_mean        0
news_sentiment_count_0          0
news_sentiment_count_1          0
news_sentiment_count_2          0
news_sentiment_mean             0
trends                          0
Null_value_news                 0
Null_value_stocktwit            0
week_day                        0
Thursday                        0
Tuesday                         0
Wednesday                       0
tweet_0_pct                     0
tweet_1_pct                     0
tweet_2_pct                     0
stocktwit_0_pc

In [ ]:
group_df = group_df.dropna()

In [ ]:
group_df.shape

(3770, 37)

In [ ]:
stock_encoder = pd.get_dummies(group_df['stock_symbol'])

In [ ]:
#stock_encoder

In [ ]:
group_df = pd.concat([group_df, stock_encoder], axis=1)

In [ ]:
#group_df

In [ ]:
#df1 = group_df[['tweet_sentiment_mean','stocktwit_sentiment_mean']].copy()

In [ ]:
#df1[df1['stocktwit_sentiment_mean']>0.5]

In [ ]:
group_df.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'Volume',
       'macd', 'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'news_sentiment_count_0',
       'news_sentiment_count_1', 'news_sentiment_count_2',
       'news_sentiment_mean', 'trends', 'Null_value_news',
       'Null_value_stocktwit', 'week_day', 'Thursday', 'Tuesday', 'Wednesday',
       'tweet_0_pct', 'tweet_1_pct', 'tweet_2_pct', 'stocktwit_0_pct',
       'stocktwit_1_pct', 'news_0_pct', 'news_1_pct', 'news_2_pct',
       'price_change', 'Open_t+1', 'direction', 'AAPL', 'AMGN', 'AXP', 'BA',
       'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM',
       'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MSFT', 'NKE', 'PG', 'TRV',
       'UNH', 'V', 'VZ', 'WBA', 'WMT'],
      dtype='object')

In [ ]:
group_df[group_df['price_change'] > 0].shape

(1857, 66)

In [ ]:
group_df[group_df['price_change'] < 0].shape

(1906, 66)

In [ ]:
group_df[group_df['price_change'] == 0].shape

(7, 66)

In [ ]:
group_df[group_df['stock_symbol']=='CAT']

,stock_symbol,Date,Open,Close,High,Low,Volume,macd,tweet_sentiment_count_0,tweet_sentiment_count_1,...,MMM,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
524,CAT,2022-03-01,187.770004,182.830002,188.220001,180.639999,5110600,-1.309002,3,10,...,0,0,0,0,0,0,0,0,0,0
525,CAT,2022-03-02,185.199997,192.610001,193.559998,185.000000,5849900,-0.598743,4,17,...,0,0,0,0,0,0,0,0,0,0
526,CAT,2022-03-03,193.320007,194.850006,196.580002,192.339996,4429700,0.068104,4,11,...,0,0,0,0,0,0,0,0,0,0
527,CAT,2022-03-08,203.869995,210.000000,214.889999,203.000000,9318600,2.078451,11,36,...,0,0,0,0,0,0,0,0,0,0
528,CAT,2022-03-09,212.570007,209.779999,213.600006,207.449997,4798300,2.679792,6,20,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,CAT,2022-12-20,232.440002,235.139999,235.720001,231.860001,2649900,-1.045581,18,37,...,0,0,0,0,0,0,0,0,0,0
650,CAT,2022-12-21,237.660004,241.729996,241.910004,237.660004,4143600,-0.445944,43,54,...,0,0,0,0,0,0,0,0,0,0
651,CAT,2022-12-22,240.149994,237.539993,240.800003,233.039993,3129900,-0.350839,19,37,...,0,0,0,0,0,0,0,0,0,0
652,CAT,2022-12-27,240.690002,243.139999,245.050003,239.720001,3381500,0.141041,23,51,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df1 = group_df.groupby(['stock_symbol'])['trends'].sum()

# groupby date and calculate the sum of columns a, b, c
#sum_by_date = df1.groupby('stock_symbol').sum()

df1

stock_symbol
AAPL    4209
AMGN    3766
AXP     3787
BA      1856
CAT     5009
CRM     3348
CSCO    3082
CVX     5242
DIS     4472
DOW     4003
GS      2771
HD      3648
HON     4153
IBM     6309
INTC    4726
JNJ     5778
JPM     4167
KO      5308
MCD     4415
MMM     5121
MSFT    5033
NKE     2395
PG      5814
TRV     1297
UNH     4917
V       4639
VZ      5773
WBA     4223
WMT     2884
Name: trends, dtype: int64

In [ ]:
df1 = group_df.groupby(['stock_symbol', 'Date'])['tweet_sentiment_count_0', 'tweet_sentiment_count_1', 'tweet_sentiment_count_2','stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1'].sum()

# groupby date and calculate the sum of columns a, b, c
sum_by_date = df1.groupby('stock_symbol').sum()

df2 = sum_by_date.sum(axis=1)
df2

<ipython-input-32-4444542777ae>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = group_df.groupby(['stock_symbol', 'Date'])['tweet_sentiment_count_0', 'tweet_sentiment_count_1', 'tweet_sentiment_count_2','stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1'].sum()


stock_symbol
AAPL    250522.0
AMGN      5243.0
AXP       5371.0
BA       41301.0
CAT      10227.0
CRM      19195.0
CSCO     11960.0
CVX      19050.0
DIS      53644.0
DOW       5418.0
GS       13234.0
HD       12386.0
HON       4722.0
IBM      10394.0
INTC     31029.0
JNJ      10451.0
JPM      22761.0
KO       12495.0
MCD      10764.0
MMM       7565.0
MSFT     64494.0
NKE      17540.0
PG        7709.0
TRV       2287.0
UNH       7380.0
V        11539.0
VZ       15144.0
WBA       7216.0
WMT      30217.0
dtype: float64

In [ ]:
df1 = group_df.groupby(['stock_symbol', 'Date'])['stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1'].sum()

# groupby date and calculate the sum of columns a, b, c
sum_by_date = df1.groupby('stock_symbol').sum()

df2 = sum_by_date[['stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1']].sum(axis=1)
df2

<ipython-input-33-19f2f507b91a>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = group_df.groupby(['stock_symbol', 'Date'])['stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1'].sum()


stock_symbol
AAPL    150387.0
AMGN      1673.0
AXP       1653.0
BA       25775.0
CAT       3450.0
CRM       8708.0
CSCO      3819.0
CVX       6555.0
DIS      30716.0
DOW       1637.0
GS        5061.0
HD        5032.0
HON       1421.0
IBM       3860.0
INTC     15665.0
JNJ       4135.0
JPM      10849.0
KO        3735.0
MCD       3640.0
MMM       2326.0
MSFT     33662.0
NKE       7492.0
PG        2294.0
TRV        661.0
UNH       2831.0
V         3984.0
VZ        6261.0
WBA       2764.0
WMT      13835.0
dtype: float64

# 1. price, macd, twitter, stocktwit---finished

#2. price, macd, twitter, stocktwit, news---finished

#3. price,macd,twitter,stocktwit,news,trends---finished

#4. price,macd,twitter,stocktwit,trends---finished

#5. price,macd,news,trends---finished

#6. price,macd,twitter---finished

#7. price,macd,stocktwit---finished

#8. price,macd,news---finished

#9. price,macd,trends---finished

#10. price,macd---finished

In [360]:
data = group_df[['stock_symbol', 'Date', 
       'Open', 'Close', 'High', 'Low', 
       'macd', 
       'Thursday', 'Tuesday', 'Wednesday', 
       'price_change', 
       'Open_t+1', 
       'direction', 
       'tweet_sentiment_count_0', 'tweet_sentiment_count_1', 
       'tweet_sentiment_count_2', 'tweet_sentiment_mean', 
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1', 
       'stocktwit_sentiment_mean', 
       #'news_sentiment_count_0', 
       #'news_sentiment_count_1', 'news_sentiment_count_2', 
       #'news_sentiment_mean', 
       #'trends', 
       #'Null_value_news',
       'Null_value_stocktwit', 
       #'AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM',
       #'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ',
       #'JPM', 'KO', 'MCD', 'MMM', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ',
       #'WBA', 'WMT'
       ]].copy()

In [361]:
data.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'macd',
       'Thursday', 'Tuesday', 'Wednesday', 'price_change', 'Open_t+1',
       'direction', 'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'Null_value_stocktwit'],
      dtype='object')

In [362]:
data.shape

(3770, 21)

In [363]:
# scale globally

# scaler = StandardScaler()

# # Group the data by stock_symbol
# numeric_columns = data.drop(['direction','Open_t+1'], axis=1).select_dtypes(include=['float64', 'int64']).columns
# scaled_df = data.copy()
# scaled_df[numeric_columns] = scaler.fit_transform(data[numeric_columns])
# grouped = scaled_df.groupby('stock_symbol')


# X_list = []
# y_list = []
# Z_list = []

# seq_length = 9

# for stock_symbol, group in grouped:
#     # Scale the data
#     scaled_x = group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1)
#     scaled_y = np.array(group['direction'])
#     scaled_z = np.array(group['Open_t+1'])

#     X = []
#     y = []
#     Z = []

#     for i in range(seq_length, len(scaled_x)):
#         X.append(scaled_x[i-seq_length:i])
#         y.append(scaled_y[i])
#         Z.append(scaled_z[i])

#     X_list.append(np.array(X))
#     y_list.append(np.array(y))
#     Z_list.append(np.array(Z))

# # Concatenate the data for all stocks
# X_train = np.concatenate(X_list, axis=0)
# y_train = np.concatenate(y_list, axis=0)
# Open_t_1_train = np.concatenate(Z_list, axis=0)



In [364]:
# scale separately but reserve stock symbol dummy

# from sklearn.preprocessing import StandardScaler
# import pandas as pd
# import numpy as np

# def reshape_train_data(data):
#   # Prepare the data
#   scaler = StandardScaler()

#     # Group the data by stock_symbol
#   grouped = data.groupby('stock_symbol')

#   X_list = []
#   y_list = []
#   Z_list = []

#   seq_length = 9

#   for stock_symbol, group in grouped:
#       # Scale the data
#       #scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
#       numeric_columns = group.drop(['direction','Open_t+1'], axis=1).select_dtypes(include=['float64', 'int64']).columns
#       scaled_x = group.copy()
#       scaled_x[numeric_columns] = scaler.fit_transform(group[numeric_columns])
#       scaled_x = scaled_x.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1)

#       scaled_y = np.array(group['direction'])
#       scaled_z = np.array(group['Open_t+1'])


#       X = []
#       y = []
#       Z = []

#       for i in range(seq_length, len(scaled_x)):
#           X.append(scaled_x[i-seq_length:i])
#           y.append(scaled_y[i])
#           Z.append(scaled_z[i])

#       X_list.append(np.array(X))
#       y_list.append(np.array(y))
#       Z_list.append(np.array(Z))

#   # Concatenate the data for all stocks
#   X_train = np.concatenate(X_list, axis=0)
#   y_train = np.concatenate(y_list, axis=0)
#   Open_t_1_train = np.concatenate(Z_list, axis=0)

#   return X_train, y_train, Open_t_1_train




In [365]:
def train_val_test_split(data):
    size=int(len(data)*0.6)
    # for train data will be collected from each country's data which index is from 0-size (60%)
    x_train = data.drop(columns=['direction']).iloc[0:size] 
    # for validation data 
    rest = data.iloc[size:]

    size2 = int(len(rest)*0.3)
    x_val = rest.drop(columns=['direction']).iloc[0:size2]
    # for test data
    x_test = rest.drop(columns=['direction']).iloc[size2:] 

    y_train = data['direction'].iloc[0:size] 
    y_val = rest['direction'].iloc[0:size2]
    y_test =  rest['direction'].iloc[size2:]

    return x_train, x_val, x_test, y_train, y_val, y_test

In [366]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def reshape_train_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])


      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_train = np.concatenate(X_list, axis=0)
  y_train = np.concatenate(y_list, axis=0)
  Open_t_1_train = np.concatenate(Z_list, axis=0)

  return X_train, y_train, Open_t_1_train




In [367]:


def reshape_val_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_val = np.concatenate(X_list, axis=0)
  y_val = np.concatenate(y_list, axis=0)
  Open_t_1_val = np.concatenate(Z_list, axis=0)

  return X_val, y_val, Open_t_1_val

  


In [368]:


def reshape_test_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 9

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction','Open_t+1'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Open_t+1'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_test = np.concatenate(X_list, axis=0)
  y_test = np.concatenate(y_list, axis=0)
  Open_t_1_test = np.concatenate(Z_list, axis=0)

  return X_test, y_test, Open_t_1_test

  


In [369]:
stock=list(set(data.stock_symbol))
stock.sort()

In [370]:

# loop each country_Region and split the data into train and test data 
X_train=[]
X_val=[]
X_test=[]
Y_train=[]
Y_val=[]
Y_test=[]

for x in stock:
    stock_data = data[data['stock_symbol']==x]
    # applied the function I created above
    x_train, x_val, x_test, y_train, y_val, y_test=train_val_test_split(stock_data) 
    X_train.append(x_train)
    X_val.append(x_val)
    X_test.append(x_test)
    Y_train.append(y_train)
    Y_val.append(y_val)
    Y_test.append(y_test)

In [371]:
X_train=pd.concat(X_train)
Y_train=pd.DataFrame(pd.concat(Y_train))
# concatenate each test dataset in X_test list and Y_test list respectively

X_val=pd.concat(X_val)
Y_val=pd.DataFrame(pd.concat(Y_val))

X_test=pd.concat(X_test)
Y_test=pd.DataFrame(pd.concat(Y_test))

In [372]:
#combine X train and Y train as train data 
train_data=pd.DataFrame()
train_data[X_train.columns]=X_train
train_data[Y_train.columns]=Y_train

val_data=pd.DataFrame()
val_data[X_val.columns]=X_val
val_data[Y_val.columns]=Y_val

test_data=pd.DataFrame()
test_data[X_test.columns]=X_test
test_data[Y_test.columns]=Y_test

In [373]:
#test_data

In [374]:
train_data.shape

(2262, 21)

In [375]:
#val_data

In [376]:
val_data.shape

(435, 21)

In [377]:
test_data.shape

(1073, 21)

In [378]:
X_train, y_train, Open_t_1_train = reshape_train_data(train_data)
X_val, y_val, Open_t_1_val = reshape_val_data(val_data)
X_test, y_test, Open_t_1_test = reshape_test_data(test_data)

In [379]:
X_train

array([[[ 0.67699848,  0.53080159,  0.65760386, ..., -0.31326146,
          0.41188415,  0.        ],
        [ 0.65297979,  0.79302584,  0.71774525, ..., -0.22079654,
          2.36610503,  0.        ],
        [ 0.96909943,  0.76727153,  0.84040317, ..., -0.13908335,
          1.13527708,  0.        ],
        ...,
        [-0.39222946, -0.10212649, -0.21524386, ..., -0.30896077,
          1.25175575,  0.        ],
        [ 0.0842749 ,  0.24906663,  0.1353191 , ...,  0.03939546,
          1.41509464,  0.        ],
        [ 0.20514393,  0.32945073,  0.21445308, ..., -0.41217743,
          0.55707945,  0.        ]],

       [[ 0.65297979,  0.79302584,  0.71774525, ..., -0.22079654,
          2.36610503,  0.        ],
        [ 0.96909943,  0.76727153,  0.84040317, ..., -0.13908335,
          1.13527708,  0.        ],
        [ 0.22141531,  0.08127484,  0.36322536, ...,  0.39635308,
          0.74669371,  0.        ],
        ...,
        [ 0.0842749 ,  0.24906663,  0.1353191 , ...,  

In [380]:
X_val.shape

(174, 9, 17)

In [381]:
X_test.shape

(812, 9, 17)

In [382]:
Open_t_1_test.shape

(812,)

In [383]:
#stock

In [384]:
#results_dict = {}

In [385]:

#build model
model = keras.Sequential()
model.add(layers.LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(layers.LSTM(100, return_sequences=False))
#model.add(layers.LSTM(256, return_sequences=False))
#model.add(layers.LSTM(256, return_sequences=False))
#model.add(layers.Dense(50))
model.add(layers.Dense(25))
model.add(layers.Dense(1,activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), #validation_split=0.2, 
                verbose=1, epochs=12, shuffle=False)

scores = model.evaluate(X_test, y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Get prediction
predicted_probabilities = model.predict(X_test)
predicted_labels = [1 if prob >= 0.5 else 0 for prob in predicted_probabilities]

Acc = accuracy_score(y_test, predicted_labels)
class_report = classification_report(y_test, predicted_labels)
class_report_lines = class_report.split('\n')
precision_0 = class_report_lines[2].split()[1]
precision_1 = class_report_lines[3].split()[1]

print(Acc, precision_0, precision_1)

    # Add the results to the dictionary
#    results_dict[x] = {'accuracy': Acc, 'precision_0': precision_0, 'precision_1': precision_1}

Epoch 1/12
63/63 [==============================] - 7s 42ms/step - loss: 0.6884 - accuracy: 0.5437 - val_loss: 0.6554 - val_accuracy: 0.7069
Epoch 2/12
63/63 [==============================] - 2s 27ms/step - loss: 0.6779 - accuracy: 0.5612 - val_loss: 0.6411 - val_accuracy: 0.7126
Epoch 3/12
63/63 [==============================] - 2s 29ms/step - loss: 0.6720 - accuracy: 0.5792 - val_loss: 0.6285 - val_accuracy: 0.7414
Epoch 4/12
63/63 [==============================] - 3s 40ms/step - loss: 0.6639 - accuracy: 0.5937 - val_loss: 0.6268 - val_accuracy: 0.7184
Epoch 5/12
63/63 [==============================] - 2s 38ms/step - loss: 0.6519 - accuracy: 0.6222 - val_loss: 0.6198 - val_accuracy: 0.6897
Epoch 6/12
63/63 [==============================] - 2s 25ms/step - loss: 0.6385 - accuracy: 0.6382 - val_loss: 0.6187 - val_accuracy: 0.6782
Epoch 7/12
63/63 [==============================] - 2s 25ms/step - loss: 0.6202 - accuracy: 0.6507 - val_loss: 0.6298 - val_accuracy: 0.6667
Epoch 8/12
63

In [ ]:
len(y_test)

841

In [ ]:
len(predicted_labels)

841

In [ ]:
test_data

,stock_symbol,Date,Open,Close,High,Low,macd,Thursday,Tuesday,Wednesday,...,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT,direction
93,AAPL,2022-10-04,145.029999,146.100006,146.220001,144.259995,-0.888134,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
94,AAPL,2022-10-05,144.070007,146.399994,147.380005,143.009995,-0.559898,0,0,1,...,0,0,0,0,0,0,0,0,0,1.0
95,AAPL,2022-10-06,145.809998,145.429993,147.539993,145.220001,-0.354758,1,0,0,...,0,0,0,0,0,0,0,0,0,0.0
96,AAPL,2022-10-11,139.899994,138.979996,141.350006,138.220001,-0.574868,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
97,AAPL,2022-10-12,139.130005,138.339996,140.360001,138.160004,-0.574481,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793,WMT,2022-12-20,142.919998,144.050003,144.639999,142.250000,-1.259160,0,1,0,...,0,0,0,0,0,0,0,0,1,1.0
3794,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,-1.058663,0,0,1,...,0,0,0,0,0,0,0,0,1,1.0
3795,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,-0.992514,1,0,0,...,0,0,0,0,0,0,0,0,1,0.0
3796,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,-0.770568,0,1,0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
# Group the DataFrame by stock_symbol
test_group = test_data.groupby("stock_symbol")

# Remove the first 7 days of data for each stock symbol
test1 = test_group.apply(lambda x: x.iloc[9:])
test1.reset_index(drop=True, inplace=True)
#test1

In [ ]:
test1

,stock_symbol,Date,Open,Close,High,Low,macd,Thursday,Tuesday,Wednesday,...,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT,direction
0,AAPL,2022-10-25,150.089996,152.339996,152.490005,149.360001,1.679308,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
1,AAPL,2022-10-26,150.960007,149.350006,151.990005,148.039993,1.640033,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
2,AAPL,2022-10-27,148.070007,144.800003,149.050003,144.130005,1.260906,1,0,0,...,0,0,0,0,0,0,0,0,0,1.0
3,AAPL,2022-11-01,155.080002,150.649994,155.449997,149.130005,1.478571,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
4,AAPL,2022-11-02,148.949997,145.029999,152.169998,145.000000,0.906723,0,0,1,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,WMT,2022-12-20,142.919998,144.050003,144.639999,142.250000,-1.259160,0,1,0,...,0,0,0,0,0,0,0,0,1,1.0
808,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,-1.058663,0,0,1,...,0,0,0,0,0,0,0,0,1,1.0
809,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,-0.992514,1,0,0,...,0,0,0,0,0,0,0,0,1,0.0
810,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,-0.770568,0,1,0,...,0,0,0,0,0,0,0,0,1,0.0


In [ ]:
test1[test1['price_change'] > 0].shape

(418, 50)

In [ ]:
test1[test1['price_change'] < 0].shape

(392, 50)

In [ ]:
test1[test1['price_change'] == 0].shape

(2, 50)

In [ ]:
test1['Prediction'] = predicted_labels
test1['Open_t_1_test'] = Open_t_1_test

In [ ]:
def dates(group):
    group['Date'] = pd.to_datetime(group['Date'])
    group = group[group['Date'] >= '2022-10-25']
    return group

test1 = test1.groupby('stock_symbol', group_keys=False).apply(dates)

In [ ]:
#test1

In [ ]:
# Select the rows where prediction is equal to 1
prediction_1 = test1[test1["Prediction"] == 1].copy()

# Compute the subtraction and sum operation on the selected rows

return_rate = ((100/prediction_1["Open"])*prediction_1["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
profit = (100/prediction_1["Open"])*prediction_1["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
#result = prediction_1["Open_t_1_test"].sub(prediction_1["Open"]).sum()


# overall_return_rate = 1
# for rate in result:
#     overall_return_rate *= (1 + rate)

# # Step 3: Convert the overall return rate back to a percentage
# overall_return_percentage = (overall_return_rate - 1) * 100

# print("Overall return rate:", overall_return_percentage, "%")


# Print the result
print(return_rate, profit)

print(sum(return_rate), sum(profit))
print(return_rate.mean(), profit.mean())

8      0.017781
9      0.075738
10    -0.022302
12     0.009608
13     0.006909
         ...   
826    0.004245
829   -0.007592
830   -0.006632
835   -0.020274
838   -0.007045
Length: 402, dtype: float64 8      1.776365
9      7.566224
10    -2.227924
12     0.959812
13     0.690213
         ...   
826    0.424045
829   -0.758472
830   -0.662498
835   -2.025349
838   -0.703751
Length: 402, dtype: float64
0.971878040193466 97.09071330604063
0.002417607065157877 0.24151918732845928


In [ ]:
test1.loc[test1['Prediction'] == 1, 'return rate'] = return_rate
test1.loc[test1['Prediction'] == 1, 'profit'] = profit

In [ ]:
#test1

In [ ]:
def calculate_profit(group):
  return_rate  = ((100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
  profit = (100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
  x = return_rate.sum()
  y = return_rate.mean()
  m = profit.sum()
  n = profit.mean()
  return x,y,m,n

# group the DataFrame by stock symbol and calculate the accuracy for each group
profit_df = prediction_1[['stock_symbol','Prediction','Open','Open_t_1_test']].groupby('stock_symbol').apply(calculate_profit).reset_index(name='profit')

# Extract the x and y values from the 'profit' column
profit_df[['return rate sum', 'return rate mean','profit sum','profit mean']] = profit_df['profit'].apply(pd.Series)

# Drop the 'profit' column
profit_df = profit_df.drop('profit', axis=1)

# print the resulting DataFrame
#profit_df

In [ ]:
profit_df['profit sum'].sum()

97.09071330604063

In [ ]:
# create a list of repeated stock symbols
# stock_symbols_repeated = np.repeat(stock, 30)
# d = pd.DataFrame({'Stock Symbol': stock_symbols_repeated, 'Predicted Labels': predicted_labels, 'Actual Labels':y_test})
# d

In [ ]:
def calculate_accuracy(group):
    return (group['Prediction'] == group['direction']).mean()

# group the DataFrame by stock symbol and calculate the accuracy for each group
accuracy_df = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(calculate_accuracy).reset_index(name='Accuracy')

# print the resulting DataFrame
#accuracy_df

In [ ]:
accuracy_df.Accuracy.mean()

0.5320197044334976

In [ ]:
# define a function to calculate the precision for a group of rows
def calculate_precision(group, label):
    tp = ((group['Prediction'] == label) & (group['direction'] == label)).sum()
    fp = ((group['Prediction'] == label) & (group['direction'] != label)).sum()
    if (tp + fp) == 0:
        return 0
    else:
        return tp / (tp + fp)

# group the DataFrame by stock symbol and calculate the precision for each group
precision_0 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 0)).reset_index(name='Precision_0')
precision_1 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 1)).reset_index(name='Precision_1')

# merge the precision DataFrames with the accuracy DataFrame
result_df = pd.merge(accuracy_df, precision_0, on='stock_symbol').merge(precision_1, on='stock_symbol')

# print the resulting DataFrame
#result_df

In [ ]:
result_df.Precision_0.mean()

0.6093523334902645

In [ ]:
result_df.Precision_1.mean()

0.5780800962438627

In [ ]:
result_df.Accuracy.mean()

0.5874384236453203

In [ ]:
# Group the data by date and calculate accuracy
#accuracy_date = test1.groupby('Date').apply(lambda x: (x['direction'] == x['Prediction']).mean()).reset_index()
#accuracy_date.columns = ['Date', 'accuracy']

# Print the accuracy for each date
#accuracy_date

In [ ]:
#accuracy_date.accuracy.mean()

In [ ]:
final_result = result_df.merge(profit_df, on=['stock_symbol'], how='left')

In [ ]:
final_result

,stock_symbol,Accuracy,Precision_0,Precision_1,return rate sum,return rate mean,profit sum,profit mean
0,AAPL,0.642857,0.818182,0.529412,-0.018831,-0.001108,-1.881233,-0.110661
1,AMGN,0.535714,0.750000,0.450000,0.027727,0.001386,2.769942,0.138497
2,AXP,0.642857,0.700000,0.611111,0.060460,0.003359,6.040003,0.335556
3,BA,0.678571,0.666667,0.684211,0.299371,0.015756,29.907203,1.574063
4,CAT,0.714286,0.714286,0.714286,0.230219,0.010963,22.998865,1.095184
5,CRM,0.642857,0.733333,0.538462,0.085576,0.006583,8.549005,0.657616
6,CSCO,0.500000,0.545455,0.470588,0.086488,0.005088,8.640141,0.508244
7,CVX,0.500000,0.545455,0.470588,-0.050782,-0.002987,-5.073166,-0.298422
8,DIS,0.607143,0.818182,0.470588,0.000139,0.000008,0.013916,0.000819
9,DOW,0.500000,0.428571,0.523810,0.054639,0.002602,5.458455,0.259926


In [ ]:
test1.to_csv('drive/My Drive/thesis/dataset/final_prediction_dataset/ [New] final_prediction_combined_model [open to open] - price+macd.csv')

In [ ]:
final_result.to_csv('drive/My Drive/thesis/dataset/final_prediction/ [New] final_prediction_combined_model [open to open] - price+macd.csv')

# benchmark

In [ ]:
# Group the DataFrame by stock_symbol
test_group = test_data.groupby("stock_symbol")

# Remove the first 7 days of data for each stock symbol
test1 = test_group.apply(lambda x: x.iloc[9:])
test1.reset_index(drop=True, inplace=True)
#test1

In [ ]:
#test1['Prediction'] = predicted_labels
test1['Open_t_1_test'] = Open_t_1_test

In [ ]:
test1

,stock_symbol,Date,Open,Close,High,Low,macd,Thursday,Tuesday,Wednesday,...,NKE,PG,TRV,UNH,V,VZ,WBA,WMT,direction,Open_t_1_test
0,AAPL,2022-10-25,150.089996,152.339996,152.490005,149.360001,1.679308,0,1,0,...,0,0,0,0,0,0,0,0,1.0,150.960007
1,AAPL,2022-10-26,150.960007,149.350006,151.990005,148.039993,1.640033,0,0,1,...,0,0,0,0,0,0,0,0,0.0,148.070007
2,AAPL,2022-10-27,148.070007,144.800003,149.050003,144.130005,1.260906,1,0,0,...,0,0,0,0,0,0,0,0,1.0,155.080002
3,AAPL,2022-11-01,155.080002,150.649994,155.449997,149.130005,1.478571,0,1,0,...,0,0,0,0,0,0,0,0,0.0,148.949997
4,AAPL,2022-11-02,148.949997,145.029999,152.169998,145.000000,0.906723,0,0,1,...,0,0,0,0,0,0,0,0,0.0,142.059998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,WMT,2022-12-20,142.919998,144.050003,144.639999,142.250000,-1.259160,0,1,0,...,0,0,0,0,0,0,0,1,1.0,144.320007
808,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,-1.058663,0,0,1,...,0,0,0,0,0,0,0,1,1.0,144.770004
809,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,-0.992514,1,0,0,...,0,0,0,0,0,0,0,1,0.0,144.039993
810,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,-0.770568,0,1,0,...,0,0,0,0,0,0,0,1,0.0,143.470001


In [ ]:

# Compute the subtraction and sum operation on the selected rows

return_rate = ((100/test1["Open"])*test1["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
profit = (100/test1["Open"])*test1["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
#result = prediction_1["Open_t_1_test"].sub(prediction_1["Open"]).sum()


# overall_return_rate = 1
# for rate in result:
#     overall_return_rate *= (1 + rate)

# # Step 3: Convert the overall return rate back to a percentage
# overall_return_percentage = (overall_return_rate - 1) * 100

# print("Overall return rate:", overall_return_percentage, "%")


# Print the result
print(return_rate, profit)

print(sum(return_rate), sum(profit))
print(return_rate.mean(), profit.mean())

0      0.003795
1     -0.021146
2      0.045340
3     -0.041530
4     -0.048259
         ...   
807    0.007794
808    0.001116
809   -0.007045
810   -0.005959
811   -0.011272
Length: 812, dtype: float64 0      0.379079
1     -2.112500
2      4.529509
3     -4.148849
4     -4.821087
         ...   
807    0.778596
808    0.111493
809   -0.703751
810   -0.595322
811   -1.126097
Length: 812, dtype: float64
0.08782067909546601 8.773294614931686
0.00010815354568407149 0.010804550018388776


In [ ]:
test1['return rate'] = return_rate
test1['profit'] = profit

In [ ]:
test1

,stock_symbol,Date,Open,Close,High,Low,macd,Thursday,Tuesday,Wednesday,...,TRV,UNH,V,VZ,WBA,WMT,direction,Open_t_1_test,return rate,profit
0,AAPL,2022-10-25,150.089996,152.339996,152.490005,149.360001,1.679308,0,1,0,...,0,0,0,0,0,0,1.0,150.960007,0.003795,0.379079
1,AAPL,2022-10-26,150.960007,149.350006,151.990005,148.039993,1.640033,0,0,1,...,0,0,0,0,0,0,0.0,148.070007,-0.021146,-2.112500
2,AAPL,2022-10-27,148.070007,144.800003,149.050003,144.130005,1.260906,1,0,0,...,0,0,0,0,0,0,1.0,155.080002,0.045340,4.529509
3,AAPL,2022-11-01,155.080002,150.649994,155.449997,149.130005,1.478571,0,1,0,...,0,0,0,0,0,0,0.0,148.949997,-0.041530,-4.148849
4,AAPL,2022-11-02,148.949997,145.029999,152.169998,145.000000,0.906723,0,0,1,...,0,0,0,0,0,0,0.0,142.059998,-0.048259,-4.821087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,WMT,2022-12-20,142.919998,144.050003,144.639999,142.250000,-1.259160,0,1,0,...,0,0,0,0,0,1,1.0,144.320007,0.007794,0.778596
808,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,-1.058663,0,0,1,...,0,0,0,0,0,1,1.0,144.770004,0.001116,0.111493
809,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,-0.992514,1,0,0,...,0,0,0,0,0,1,0.0,144.039993,-0.007045,-0.703751
810,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,-0.770568,0,1,0,...,0,0,0,0,0,1,0.0,143.470001,-0.005959,-0.595322


In [ ]:
def calculate_profit(group):
  return_rate  = ((100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
  profit = (100/group["Open"])*group["Open_t_1_test"]*(1-0.001)-100*(1+0.001)
  x = return_rate.sum()
  y = return_rate.mean()
  m = profit.sum()
  n = profit.mean()
  return x,y,m,n

# group the DataFrame by stock symbol and calculate the accuracy for each group
profit_df = test1[['stock_symbol','Open','Open_t_1_test']].groupby('stock_symbol').apply(calculate_profit).reset_index(name='profit')

# Extract the x and y values from the 'profit' column
profit_df[['return rate sum', 'return rate mean','profit sum','profit mean']] = profit_df['profit'].apply(pd.Series)

# Drop the 'profit' column
profit_df = profit_df.drop('profit', axis=1)

# print the resulting DataFrame
profit_df

,stock_symbol,return rate sum,return rate mean,profit sum,profit mean
0,AAPL,-0.200829,-0.007172,-20.062835,-0.716530
1,AMGN,-0.044444,-0.001587,-4.439930,-0.158569
2,AXP,-0.030332,-0.001083,-3.030210,-0.108222
3,BA,0.246969,0.008820,24.672209,0.881150
4,CAT,0.191353,0.006834,19.116228,0.682722
5,CRM,-0.256612,-0.009165,-25.635571,-0.915556
6,CSCO,0.035869,0.001281,3.583275,0.127974
7,CVX,-0.029553,-0.001055,-2.952380,-0.105442
8,DIS,-0.219556,-0.007841,-21.933690,-0.783346
9,DOW,0.021732,0.000776,2.171041,0.077537


In [ ]:
profit_df['profit sum'].sum()

8.773294614931686

In [ ]:
test1.to_csv('drive/My Drive/thesis/dataset/final_prediction_dataset/ [New] final_prediction_combined_model [open to open] - benchmark.csv')

In [ ]:
profit_df.to_csv('drive/My Drive/thesis/dataset/final_prediction/ [New] final_prediction_combined_model [open to open] - benchmark.csv')